In [33]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt  # Visualization
import warnings  # Supress warnings
import os
import random

In [34]:

warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from keras.models import load_model

In [35]:

# Seed value
SEED = 2023
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED'] = str(SEED)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
random.seed(SEED)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(SEED)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(SEED)

# 5. Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [60]:
df = pd.read_csv("./files/cycl_m.csv", index_col=0)
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
df

,Rainfall,Depth_to_Groundwater,Temperature,Drainage_Volume,River_Hydrometry,year,month,quarter,season,month_sin,month_cos,quarter_sin,quarter_cos,season_sin,season_cos
Date,,,,,,,,,,,,,,,
2009-01-31,0.032915,0.216100,0.068714,0.726750,0.396506,2009,1,1,1,5.000000e-01,8.660254e-01,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17
2009-02-28,0.073145,0.296982,0.074975,0.936674,0.508567,2009,2,1,1,8.660254e-01,5.000000e-01,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17
2009-03-31,0.178683,0.356360,0.236094,0.627008,0.469107,2009,3,1,2,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17,1.224647e-16,-1.000000e+00
2009-04-30,0.085162,0.410094,0.413676,0.693265,0.407926,2009,4,2,2,8.660254e-01,-5.000000e-01,1.224647e-16,-1.000000e+00,1.224647e-16,-1.000000e+00
2009-05-31,0.149425,0.428197,0.647873,0.400633,0.201392,2009,5,2,2,5.000000e-01,-8.660254e-01,1.224647e-16,-1.000000e+00,1.224647e-16,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29,0.097179,0.583554,0.225748,0.736863,0.420133,2020,2,1,1,8.660254e-01,5.000000e-01,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17
2020-03-31,0.366771,0.605731,0.255726,0.721515,0.632459,2020,3,1,2,1.000000e+00,6.123234e-17,1.000000e+00,6.123234e-17,1.224647e-16,-1.000000e+00
2020-04-30,0.217346,0.615560,0.421868,0.896287,0.475913,2020,4,2,2,8.660254e-01,-5.000000e-01,1.224647e-16,-1.000000e+00,1.224647e-16,-1.000000e+00


In [61]:
features_cos = ['Rainfall', 'Depth_to_Groundwater', 'Temperature', 'Drainage_Volume', 'River_Hydrometry', 'day_cos',
                'month_cos', 'week_of_year_cos', 'quarter_cos', 'season_cos']
delete = [feature for feature in df.columns if feature not in features_cos]
df_cos = df.drop(delete, axis=1)

In [62]:
features_new = ['Depth_to_Groundwater', 'Drainage_Volume', 'River_Hydrometry', 'month_cos', 'week_of_year_cos', 'quarter_cos']
features_3 = ['Rainfall', 'Depth_to_Groundwater', 'Drainage_Volume', 'River_Hydrometry', 'month_cos', 'quarter_cos']
delete3 = [feature for feature in df_cos.columns if feature not in features_3]
df_new = df_cos.drop(delete3, axis=1)

In [63]:
df = df_new
df

,Rainfall,Depth_to_Groundwater,Drainage_Volume,River_Hydrometry,month_cos,quarter_cos
Date,,,,,,
2009-01-31,0.032915,0.216100,0.726750,0.396506,8.660254e-01,6.123234e-17
2009-02-28,0.073145,0.296982,0.936674,0.508567,5.000000e-01,6.123234e-17
2009-03-31,0.178683,0.356360,0.627008,0.469107,6.123234e-17,6.123234e-17
2009-04-30,0.085162,0.410094,0.693265,0.407926,-5.000000e-01,-1.000000e+00
2009-05-31,0.149425,0.428197,0.400633,0.201392,-8.660254e-01,-1.000000e+00
...,...,...,...,...,...,...
2020-02-29,0.097179,0.583554,0.736863,0.420133,5.000000e-01,6.123234e-17
2020-03-31,0.366771,0.605731,0.721515,0.632459,6.123234e-17,6.123234e-17
2020-04-30,0.217346,0.615560,0.896287,0.475913,-5.000000e-01,-1.000000e+00


In [15]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [16]:
from tensorboard.plugins.hparams import api as hp

In [64]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 96, 128]))
HP_N_STEPS_IN = hp.HParam('n_steps_in', hp.Discrete([3, 5, 7, 10, 15]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2, 0.3, 0.4]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'nadam']))

METRIC_RMSE = 'RootMeanSquaredError'

logdir = 'logs/hparam_tuning_multi'
with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_N_STEPS_IN, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_RMSE, display_name='RMSE')]
    )


In [65]:
N_FEATURES  = 6  # multivariate
#N_STEPS_IN  = 5 # static window
N_STEPS_OUT = 1  # single-step

In [66]:
def df_to_X_y(df, hparams):
    N_STEPS_IN  = hparams[HP_N_STEPS_IN]
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - N_STEPS_IN):
        row = [r for r in df_as_np[i:i + N_STEPS_IN]]
        X.append(row)
        label = df_as_np[i + N_STEPS_IN][1]
        y.append(label)
    return np.array(X), np.array(y)

def get_sets(df, hparams):
    X1, y1 = df_to_X_y(df, hparams)
    N_STEPS_IN  = hparams[HP_N_STEPS_IN]
    TRAIN_SIZE = int((len(df.index)-N_STEPS_IN) * 0.5)
    VAL_SIZE = int((len(df.index)-N_STEPS_IN) * 0.2)

    X_train1, y_train1 = X1[:TRAIN_SIZE], y1[:TRAIN_SIZE]
    X_val1, y_val1 = X1[TRAIN_SIZE:TRAIN_SIZE + VAL_SIZE], y1[TRAIN_SIZE:TRAIN_SIZE + VAL_SIZE]
    X_test1, y_test1 = X1[TRAIN_SIZE + VAL_SIZE:], y1[TRAIN_SIZE + VAL_SIZE:]
    return X1, y1, X_train1, y_train1, X_val1, y_val1, X_test1, y_test1

In [45]:
def init_model(hparams):
    # Use initializers to initialize model parameters with the same values
    INITIALIZER_GLOROT_UNIFORM = tf.keras.initializers.GlorotUniform(seed=SEED)
    INITIALIZER_ORTHOGONAL = tf.keras.initializers.Orthogonal(gain=1.0, seed=SEED)
    model = Sequential()
    model.add(InputLayer((hparams[HP_N_STEPS_IN], N_FEATURES)))
    model.add(LSTM(units=hparams[HP_NUM_UNITS],
                   kernel_initializer=INITIALIZER_GLOROT_UNIFORM,
                   recurrent_initializer=INITIALIZER_ORTHOGONAL))
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    model.add(Dense(units=8, activation='relu', kernel_initializer=INITIALIZER_GLOROT_UNIFORM))
    model.add(Dense(units=N_STEPS_OUT,
                    kernel_initializer=INITIALIZER_GLOROT_UNIFORM,
                    activation='linear'))
    return model

In [57]:
def train_model(df, hparams):
    X1, y1, X_train1, y_train1, X_val1, y_val1, X_test1, y_test1 = get_sets(df, hparams)
    model = init_model(hparams)
    model.compile(loss=MeanSquaredError(), optimizer=hparams[HP_OPTIMIZER], metrics=[RootMeanSquaredError()])
    model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=15, verbose=0, callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ])
    return model

In [47]:
def train_test_model(df, hparams):
    X1, y1, X_train1, y_train1, X_val1, y_val1, X_test1, y_test1 = get_sets(df, hparams)
    model = train_model(df, hparams)
    _, rmse = model.evaluate(X_test1, y_test1)
    return  rmse

In [48]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        rmse = train_test_model(df, hparams)
        tf.summary.scalar(METRIC_RMSE, rmse, step=10)

In [67]:
session_num = 0

for n_steps_in in HP_N_STEPS_IN.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for dropout_rate in HP_DROPOUT.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_N_STEPS_IN: n_steps_in,
                    HP_NUM_UNITS: num_units,
                    HP_DROPOUT: dropout_rate,
                    HP_OPTIMIZER: optimizer,
                }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/lstm/hparam_tuning_3/' + run_name, hparams)
                session_num += 1


--- Starting trial: run-0
{'n_steps_in': 3, 'num_units': 64, 'dropout': 0.1, 'optimizer': 'adam'}
2/2 [==============================] - 0s 4ms/step - loss: 0.0182 - root_mean_squared_error: 0.1349
--- Starting trial: run-1
{'n_steps_in': 3, 'num_units': 64, 'dropout': 0.1, 'optimizer': 'nadam'}
2/2 [==============================] - 0s 5ms/step - loss: 0.0263 - root_mean_squared_error: 0.1620
--- Starting trial: run-2
{'n_steps_in': 3, 'num_units': 64, 'dropout': 0.2, 'optimizer': 'adam'}
2/2 [==============================] - 0s 4ms/step - loss: 0.0182 - root_mean_squared_error: 0.1349
--- Starting trial: run-3
{'n_steps_in': 3, 'num_units': 64, 'dropout': 0.2, 'optimizer': 'nadam'}
2/2 [==============================] - 0s 5ms/step - loss: 0.0263 - root_mean_squared_error: 0.1620
--- Starting trial: run-4
{'n_steps_in': 3, 'num_units': 64, 'dropout': 0.3, 'optimizer': 'adam'}
2/2 [==============================] - 0s 6ms/step - loss: 0.0182 - root_mean_squared_error: 0.1349
--- Star

In [70]:
%tensorboard --logdir logs/lstm/hparam_tuning_3 --port 6014

Launching TensorBoard...

In [69]:
import winsound
winsound.Beep(2500,1000)